In [78]:
import matplotlib.pyplot as plt
import numpy as np
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import LSTM, Dense,TimeDistributed
import tensorflow as tf
import mne
import gc

Konfiguracja modelu

In [79]:
train_files=['p10_Record2.edf', 'p11_Record1.edf', 'p11_Record2.edf', 'p11_Record3.edf', 'p11_Record4.edf', 'p12_Record1.edf', 'p12_Record2.edf', 'p12_Record3.edf', 'p13_Record1.edf', 'p13_Record2.edf', 'p13_Record3.edf', 'p13_Record4.edf', 'p14_Record1.edf', 'p14_Record2.edf', 'p14_Record3.edf', 'p15_Record1.edf', 'p15_Record2.edf', 'p15_Record3.edf', 'p15_Record4.edf']
test_file= 'p10_Record1.edf'
frame_size=1000
split_ratio=0.8
checkpoint_filepath="./checkpoint"
#Przełączniki którymi można się bawić
units_of_LSTM=19
number_of_epochs=1
batch_size=1

Definiowanie modelu:
    LSTM units: ile równolegle ma być komórek typu LSTM 
    Return_sequences: czy lstm ma zwracać sekwencje
    Input_shape: (ilość elementów w jednej ramce,liczba cech)
    Dense: warstwa najprostszych modeli neuronów by złożyć nam neurony lstm w jedno
Kompilowanie 
    loss: Funkcja oceny

In [80]:
model = Sequential([
    TimeDistributed(Dense(19),input_shape=(frame_size, 19)),
    LSTM(units=units_of_LSTM,return_sequences=True),
    Dense(units=19)
])
model.compile(optimizer='adam', loss='mean_squared_error')
model_checkpoint=ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    save_best_only=True,
    monitor='val_loss',
    mode='auto'
)
model.summary()

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_15 (TimeD  (None, 1000, 19)          380       
 istributed)                                                     
                                                                 
 lstm_18 (LSTM)              (None, 1000, 20)          3200      
                                                                 
 dense_34 (Dense)            (None, 1000, 19)          399       
                                                                 
Total params: 3979 (15.54 KB)
Trainable params: 3979 (15.54 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Odczytanie pliku.
Wybranie dziewietnastu kanałów.
Obliczenie rozmiaru ramki.
Przyporządkowanie ramce kolejnej ramki.
Usunięcie ostatniej ramki i ramki jej przyporządkowanej
Trenowanie modelu:
    batch_size=rozmiar użytych próbek
    epochs= liczba epok

In [81]:
for file in train_files:
    edf=mne.io.read_raw_edf(file,preload=True)
    raw_data=mne.make_fixed_length_epochs(edf,2)
    raw_data=raw_data.get_data()
    x=tf.transpose(tf.convert_to_tensor(raw_data),perm=[0,2,1])
    
    data_shifted = edf.get_data()[:, 1:]
    times_shifted = edf.times[1:]
    
    info = mne.create_info(ch_names=edf.ch_names, sfreq=edf.info['sfreq'], ch_types='eeg')
    edf = mne.io.RawArray(data_shifted, info, first_samp=edf.first_samp + 1)   
    raw_data=mne.make_fixed_length_epochs(edf,2)
    raw_data=raw_data.get_data()
    y=tf.transpose(tf.convert_to_tensor(raw_data),perm=[0,2,1])
    
    length=raw_data.shape[0]-1
    x=x[:y.shape[0]-1,:,:19]
    y=y[:-1,:,:19]
    dataset = tf.data.Dataset.from_tensor_slices((x,y))
    
    del raw_data
    split = int(split_ratio * length)
    train_dataset = dataset.take(split).batch(batch_size)
    val_dataset = dataset.skip(split).batch(batch_size)
    gc.collect()
    model.fit(train_dataset, batch_size=batch_size, epochs=number_of_epochs,use_multiprocessing=True,callbacks=[model_checkpoint],validation_data=val_dataset)

Extracting EDF parameters from C:\Users\huber\PycharmProjects\pythonProject5\p10_Record2.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 5399499  =      0.000 ... 10798.998 secs...
Not setting metadata
5399 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 5399 events and 1000 original time points ...
0 bad epochs dropped
Creating RawArray with float64 data, n_channels=19, n_times=5399499
    Range : 1 ... 5399499 =      0.002 ... 10798.998 secs
Ready.
Not setting metadata
5399 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 5399 events and 1000 original time points ...
0 bad epochs dropped
1623/4318 [==========>...................] - ETA: 12:55 - loss: 1.7511e-08

KeyboardInterrupt: 

Odczyt i testowanie modelu

In [ ]:
model.load_weights(checkpoint_filepath)
train=mne.io.read_raw_edf(test_file)
train=train.get_data()
do_testowania_array=np.array(train[:19,:frame_size])

Predykcja na podstawie modelu

In [ ]:
do_testowania_array=do_testowania_array.T
wynik = model.predict([do_testowania_array.tolist()])

Rysowanie wykresu predykcji i oryginalnej wartości

In [ ]:
wynik=wynik[0]
wynik=wynik.T
for kanal in range(19):
    plt.plot(train[kanal][frame_size:frame_size*2])
    plt.plot(wynik[kanal])
    plt.show()
model.save("model")